In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Try reading the actual file (without '._' prefix)
# sample_test = pd.read_csv(r"/content/drive/MyDrive/Amazon ML Challenge/student_resource 3/dataset/sample_test.csv", encoding='ISO-8859-1')
# smaple_test_out = pd.read_csv(r"/content/drive/MyDrive/Amazon ML Challenge/student_resource 3/dataset/sample_test.csv", encoding='ISO-8859-1')
# smaple_test_out_fail = pd.read_csv(r"/content/drive/MyDrive/Amazon ML Challenge/student_resource 3/dataset/sample_test_out_fail.csv", encoding='ISO-8859-1')
test =  pd.read_csv(r"/content/drive/MyDrive/66e31d6ee96cd_student_resource_3/part_7.csv", encoding='ISO-8859-1')
# train  =  pd.read_csv(r"/content/drive/MyDrive/Amazon ML Challenge/student_resource 3/dataset/train.csv", encoding='ISO-8859-1')

In [ ]:
# print(f" train data : {train.shape}")
print(f" test data : {test.shape}")
# print(f" sample_test data : {sample_test.shape}")
# print(f" smaple_test_out data : {smaple_test_out.shape}")
# print(f" smaple_test_out_fail data : {smaple_test_out_fail.shape}")

 test data : (8199, 4)


In [ ]:
pip install requests pillow

In [ ]:
# Install Tesseract
!sudo apt-get install tesseract-ocr

# Install pytesseract
!pip install pytesseract

# Install the Pillow library to handle images
!pip install pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import pandas as pd
import json
from PIL import Image, ImageEnhance
import pytesseract
from io import BytesIO
import requests
import re
import numpy as np
from skimage.restoration import denoise_tv_chambolle
import cv2
from skimage.transform import rotate

def resize_image(image, base_width):
    w_percent = (base_width / float(image.size[0]))
    h_size = int((float(image.size[1]) * float(w_percent)))
    return image.resize((base_width, h_size), Image.LANCZOS)

def denoise_image(image):
    image_np = np.array(image)
    denoised_np = denoise_tv_chambolle(image_np, weight=0.1)
    return Image.fromarray((denoised_np * 255).astype(np.uint8))

def adaptive_threshold(image):
    image_np = np.array(image)
    if len(image_np.shape) == 2:
        gray = image_np
    else:
        gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return Image.fromarray(thresh)

def deskew_image(image):
    image_np = np.array(image)
    coords = np.column_stack(np.where(image_np > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    rotated = rotate(image_np, angle, resize=True)
    return Image.fromarray((rotated * 255).astype(np.uint8))

def enhance_sharpness(image):
    enhancer = ImageEnhance.Sharpness(image)
    return enhancer.enhance(2)

def remove_lines(image):
    return image.filter(ImageFilter.MedianFilter(size=3))

def adaptive_contrast(image):
    gray_image = image.convert('L')
    mean_brightness = np.mean(np.array(gray_image))
    enhancer = ImageEnhance.Contrast(image)
    factor = 3.0 if mean_brightness < 128 else 1.2
    return enhancer.enhance(factor)

def preprocess_image(image):
    image = image.convert('L')
    skew_angle = detect_skew(image)
    if abs(skew_angle) > 5:
        image = deskew_image(image)
    image = adaptive_contrast(image)
    image = image.point(lambda p: p > 128 and 255)
    return image

def detect_skew(image):
    image_np = np.array(image)
    edges = cv2.Canny(image_np, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
    angles = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            angles.append(angle)
    return np.median(angles) if angles else 0

def process_image(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        preprocessed_img = preprocess_image(img)
        extracted_text_tesseract = pytesseract.image_to_string(preprocessed_img)
        return extracted_text_tesseract
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the image from URL: {url}. Error: {e}")
    except OSError as e:
        print(f"Error opening or processing the image from URL: {url}. Error: {e}")
    return ""

def extract_values_for_entity(text, entity_name):
    # Define a mapping from unit abbreviation to full unit names
    unit_mapping = {
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'lb': 'pound', 'oz': 'ounce', 'ton': 'ton',
        'G': 'gram', 'KG': 'kilogram', 'MG': 'milligram', 'LB': 'pound', 'OZ': 'ounce', 'TON': 'ton',
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard',
        'CM': 'centimetre', 'M': 'metre', 'MM': 'millimetre', 'FT': 'foot', 'IN': 'inch', 'YD': 'yard',
        'V': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt','v': 'volt', 'KV': 'kilovolt', 'MV': 'millivolt','kV': 'kilovolt', 'mV': 'millivolt',
        'W': 'watt', 'kW': 'kilowatt','w': 'watt', 'KW': 'kilowatt','kw': 'kilowatt',
        'L': 'litre', 'ml': 'millilitre', 'gal': 'gallon', 'pt': 'pint', 'cup': 'cup',
        'l': 'litre', 'ML': 'millilitre', 'GAL': 'gallon', 'PT': 'pint', 'Cup': 'cup'

    }
    pattern = r'(\d+\.?\d*)\s?([a-zA-Z]+)'
    matches = re.findall(pattern, text)
    for match in matches:
        value, unit_abbr = match
        full_unit = unit_mapping.get(unit_abbr, None)
        if full_unit:
            return f"{value} {full_unit}"
    return None

def process_dataset(csv_file):
    df = pd.read_csv(csv_file)

    results = []


    for i, row in df.iterrows():
        image_url = row['image_link']
        image_id = row['index']
        entity_name = row['entity_name']
        print(f"Processing image {i + 1} with ID: {image_id}")
        extracted_text_tesseract = process_image(image_url)
        extracted_value_with_unit_tesseract = extract_values_for_entity(extracted_text_tesseract, entity_name)
        print(f"Extracted value for '{entity_name}' using Tesseract: {extracted_value_with_unit_tesseract}")

        results.append({
            'index': image_id,
            'entity_name': entity_name,
            'extracted_value_tesseract': extracted_value_with_unit_tesseract
        })

    with open('extracted_results.json', 'w') as f:
        json.dump(results, f, indent=4)

csv_file_path = '/content/drive/MyDrive/66e31d6ee96cd_student_resource_3/part_7.csv'  # Replace with the actual path to your train.csv file
process_dataset(csv_file_path)


Streaming output truncated to the last 5000 lines.
Extracted value for 'width' using Tesseract: 320 millimetre
Processing image 5701 with ID: 54942
Extracted value for 'height' using Tesseract: None
Processing image 5702 with ID: 54943
Extracted value for 'height' using Tesseract: 14.57 inch
Processing image 5703 with ID: 54944
Extracted value for 'height' using Tesseract: 00 gram
Processing image 5704 with ID: 54945
Extracted value for 'depth' using Tesseract: 00 gram
Processing image 5705 with ID: 54946
Extracted value for 'depth' using Tesseract: None
Processing image 5706 with ID: 54947
Extracted value for 'width' using Tesseract: None
Processing image 5707 with ID: 54948
Extracted value for 'height' using Tesseract: None
Processing image 5708 with ID: 54949
Extracted value for 'height' using Tesseract: 9.05 inch
Processing image 5709 with ID: 54950
Extracted value for 'maximum_weight_recommendation' using Tesseract: None
Processing image 5710 with ID: 54951
Extracted value for 'de

In [ ]:
import json
import pandas as pd

# Step 1: Load the original dataset from CSV using pandas
original_csv_path = '/content/drive/MyDrive/Amazon ML Challenge/student_resource 3/dataset/train.csv'
original_df = pd.read_csv(original_csv_path)

# Step 2: Load the extracted data (from the JSON file we generated earlier)
with open('/content/extracted_results.json', 'r') as file:
    extracted_data = json.load(file)

# Step 3: Convert the CSV into the same structure as the extracted JSON data
# Flatten data, keeping value and unit combined in the same field
def flatten_original_data(df):
    flattened = []
    for _, row in df.iterrows():
        # Check if 'entity_value' exists in the dataframe
        if 'entity_value' in row:
            # Group ID and the entity value (assuming the entity value column contains the combined value and unit)
            flattened.append((row['group_id'], str(row['entity_value'])))
    return flattened

# Step 4: Convert the extracted JSON data into a similar flattened format
def flatten_extracted_data(data):
    flattened = []
    for entry in data:
        image_id = entry['index']
        extracted_value = entry.get('extracted_value_tesseract', None)  # Use extracted value from Tesseract
        if extracted_value:  # Only include if the value is not None
            flattened.append((image_id, extracted_value))
    return flattened

# Step 5: Flatten both datasets
original_flattened = flatten_original_data(original_df)
extracted_flattened = flatten_extracted_data(extracted_data)
print(original_flattened)
print(extracted_flattened)

# Step 6: Create sets for comparison (set of tuples for true and predicted values)
true_values = set(original_flattened)  # Ground truth from the original dataset
predicted_values = set(extracted_flattened)  # Extracted values from the processed JSON data

# Step 7: Calculate Precision, Recall, and F1 Score
# True Positive: Correctly extracted values
true_positive = len(true_values & predicted_values)  # Intersection of true and predicted sets
false_positive = len(predicted_values - true_values)  # Predicted values not in the true set
false_negative = len(true_values - predicted_values)  # True values not in the predicted set

# Calculate precision, recall, and F1 score
precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Step 8: Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
